**Generative Adversarial Networks in Keras**

First let's import all the necessary libraries
In this case Keras is running in a Tensorflow backend

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Merge
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
from keras.layers.core import Reshape
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
import random
from keras.optimizers import SGD
import pandas as pd

Using TensorFlow backend.


Now let's import MNIST data

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In this step, each digit type is adjusted to 'float32'

In [3]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

The following step selects a small sample from MNIST so that you can do the computation even without a GPU
After that, we add noise to data, using a random normal distribution with mean equal to zero and standard deviation equal to 1

In [4]:
n=60

x_train=x_train[0:n]
x_test=x_test[n:n+n]

noise_factor = 0.1
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

Reshape inputs for the Neural Network that will be composed of real data and data with noise addded

In [5]:
x_train=x_train.reshape((n,28,28,1))
x_test=x_test.reshape((n,28,28,1))
x_train_noisy = x_train_noisy.reshape((n,28,28,1))
x_test_noisy = x_test_noisy.reshape((n,28,28,1))

x_train_noisy=np.concatenate([x_train_noisy,x_train]).reshape(2*n,28,28,1)
x_train=np.concatenate([x_train,x_train]).reshape(2*n,28,28,1)

We select a random sample from real data + noisy data and get one-hot encodes for y_train

In [6]:
np.random.seed(200)
sel=random.sample(range(0,x_train.shape[0]), 100)
y_train=pd.get_dummies(y_train[sel])
x_train_noisy=x_train_noisy[sel]

We set the hyperparameters

In [7]:
batch_size = 30
nb_classes = 10 # 10 classes for y_train (0,1,2,3,4,5,6,7,8,9)
img_rows, img_cols = 28, 28 #sqrt of 784
nb_filters = 32
pool_size = (2, 2)
kernel_size = (3, 3)
input_shape=(28,28,1)
learning_rate = 0.008
decay_rate = 5e-5
momentum = 0.9

Now we develop the Generative part of the GAN using Keras and compile the model

In [8]:
sgd = SGD(lr=learning_rate,momentum=momentum, decay=decay_rate, nesterov=False)


input_img = Input(shape=(28, 28, 1))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
generator = Model(input_img, decoded)
generator.compile(loss='mean_squared_error', optimizer=sgd,metrics = ['accuracy'])
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 32)          9248      
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 32)        0         
__________

Next, we create the Discriminator part of the GAN, using model.add instead of x=layer(x) to diversify our knowledge

In [9]:
discriminator = Sequential()
discriminator.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]),
                        padding='valid',
                        input_shape=input_shape))
discriminator.add(Activation('relu'))
discriminator.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
discriminator.add(Activation('relu'))
discriminator.add(MaxPooling2D(pool_size=pool_size))
discriminator.add(Dropout(0.25))
discriminator.add(Flatten())
discriminator.add(Dense(128))
discriminator.add(Activation('relu'))
discriminator.add(Dropout(0.5))
discriminator.add(Dense(10))
discriminator.add(Activation('softmax'))
discriminator.compile(loss='categorical_crossentropy', optimizer=sgd,metrics = ['accuracy'])
discriminator.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
__________

In the following cell, we'll create a function that will allow us to freeze the training of the Generator (OR NOT) so that we can try different update strategies with Discriminator and Generator

In [10]:
def trainable(net, val):
    net.trainable = val
    for k in net.layers:
       k.trainable = val
trainable(generator, False)

gan_input = Input(batch_shape=(None, 28,28,1))

gan_level2 = discriminator(generator([gan_input]))

We compile the GAN = Discriminator(Generator(input))

In [11]:
GAN = Model(gan_input, gan_level2)
GAN.compile(loss='mean_squared_error', optimizer=sgd,metrics = ['accuracy'])

Now we will update Discriminator and Generator asynchronously in the proportion 1:1

In [12]:
nb_epochs=2
rate=1 #proportion of update of Discriminator:Generator
for i in range(0,nb_epochs):
    print('Discriminator epoch', i)
    discriminator.fit(x_train_noisy,np.array(y_train),
                    epochs=nb_epochs,
                    batch_size=30,verbose=1)
    print('Generator')
    GAN.fit(x_train_noisy,np.array(y_train),
            batch_size=30, epochs=nb_epochs,verbose=1)

Discriminator epoch 0
Epoch 1/2
100/100 [==============================] - 0s - loss: 2.3213 - acc: 0.0500     
Epoch 2/2
100/100 [==============================] - 0s - loss: 2.3130 - acc: 0.1100     
Generator
Epoch 1/2
100/100 [==============================] - 0s - loss: 0.0899 - acc: 0.1400     
Epoch 2/2
100/100 [==============================] - 0s - loss: 0.0894 - acc: 0.1400     
Discriminator epoch 1
Epoch 1/2
100/100 [==============================] - 0s - loss: 2.2793 - acc: 0.0900     
Epoch 2/2
100/100 [==============================] - 0s - loss: 2.2266 - acc: 0.1800     
Generator
Epoch 1/2
100/100 [==============================] - 0s - loss: 0.0893 - acc: 0.1700     
Epoch 2/2
100/100 [==============================] - 0s - loss: 0.0893 - acc: 0.1700     


Now we reset the variables that were one-hot encoded to calculate accuracy in training and test set

In [13]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train=y_train[0:n]
y_train=np.concatenate([y_train,y_train])[sel]
y_test=y_test[0:n]
y_test=np.concatenate([y_test,y_test])[sel]
x_test_noisy=x_test_noisy[0:n]
x_test_noisy=np.concatenate([x_test_noisy,x_test_noisy])[sel]

In [ ]:
print('Accuracy Train:',1-np.count_nonzero(y_train-np.argmax(GAN.predict(x_train_noisy),axis=1))/n)

print('Accuracy Test:',1-np.count_nonzero(y_test-np.argmax(GAN.predict(x_test_noisy),axis=1))/n)